Import packages and load rotation functions

In [5]:
# import the necessary packages
#Life is good
from PIL import Image
import pytesseract
import numpy as np
#from pytesseract import image_to_string
#from pytesseract import image_to_boxes
#from pytesseract import image_to_data
import argparse
import cv2
import os

def orientationScore(image,skipStep = 10):
    totValue = 0
    for row in range(len(image)):
        rowVal = 0
        for col in range(0,len(image[0]),skipStep):
            rowVal += image[row][col]
        totValue += rowVal*rowVal
    return totValue

def findOrient(image,angleRange = 5,skipStep = 10):
    maxValue = -1
    maxRot  = 0
    imageRows, imageCols = image.shape
    accuracy = 0.1



    for i in range(-1, angleRange*2*int(1/accuracy)):
        #rot = i
        rot = (round(i / 2 - 0.4) + 1) * ((-1) * (i % 2) + ((i + 1) % 2)) * 0.1
        #print rot
        # print("Rotation: ",rot,". i: ",i)
        M = cv2.getRotationMatrix2D((imageCols / 2, imageRows / 2), rot, 1)
        dst = cv2.warpAffine(image, M, (imageCols, imageRows), borderValue=(255, 255, 255))

        value = orientationScore(255-dst,skipStep = 10)
        if value>maxValue:
            maxValue = value
            maxRot = rot
        print "Current angle: ",rot,". Current value: ", value,". Max value: ",maxValue,". Max rot: ",maxRot
        #cv2.imshow("Rotated Image: ",dst)
        #cv2.waitKey(0)

    M = cv2.getRotationMatrix2D((imageCols / 2, imageRows / 2), maxRot, 1)
    dst = cv2.warpAffine(image, M, (imageCols, imageRows), borderValue=(255, 255, 255))
    #cv2.imshow("Rotated Image: ", dst)
    #cv2.waitKey(0)
    return dst

Specify the paramaters of the OCR run

In [6]:
args = {}
args["image"] = "/home/dries/devel/PraelexisOCR/Code/RealScan.jpg"
args["preprocess"] = ""

Main code

In [12]:

# construct the argument parse and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--image", required=True,help="path to input image to be OCR'd")
# ap.add_argument("-p", "--preprocess", type=str, default="thresh",	help="type of preprocessing to be done")
# args = vars(ap.parse_args())s


# load the example image and convert it to grayscale
image = cv2.imread(args["image"])

# cv2.imshow("Image", image)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# check to see if we should apply thresholding to preprocess the
# image
if args["preprocess"] == "thresh":
    gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

# make a check to see if median blurring should be done to remove
# noise
elif args["preprocess"] == "blur":
    gray = cv2.medianBlur(gray, 1)

#gray = findOrient(gray,angleRange = 0,skipStep = 10)

# write the grayscale image to disk as a temporary file so we can
# apply OCR to it
#filename = "{}.png".format(os.getpid())
#cv2.imwrite(filename, gray)

# load the image as a PIL/Pillow image, apply OCR, and then delete
# the temporary file
#text = pytesseract.image_to_string(gray,boxes=False)
text = pytesseract.image_to_string(gray,boxes=False)
data = pytesseract.image_to_string(gray,boxes=True)

lines = data.split('\n')

print "Text: ", text

array = []


yLen = len(gray)
newPage = gray.copy()

for i in range(len(newPage)):
    for j in range(len(newPage[0])):
        newPage[i][j] = 255

for line in lines:
    entry = line.split(' ')
    array.append(entry)
    cv2.rectangle(gray, (int(entry[1]), yLen-int(entry[2])), (int(entry[3]), yLen-int(entry[4])), (0,0,0), 4)
    font = cv2.FONT_HERSHEY_SIMPLEX
    if ord(entry[0])<128:
        #print ord(entry[0])
        char = str(entry[0])
        #print(char)
        
        xSize = 0.5*(int(entry[3])-int(entry[1]))
        ySize = 0.5*(int(entry[4])-int(entry[2]))
        
        size = xSize
        
        if ySize>xSize:
            size = ySize
        
        fondSize = size/15.0
        loc = (int(entry[1])+int(0.5*(int(entry[3])-int(entry[1]))),yLen-int(entry[2])+int(0.5*(int(entry[4])-int(entry[2]))))
        
        print char," ",fondSize, " ", loc
        
        
        cv2.putText(newPage,char,loc,font, fondSize,(0,0,255))

#data = pytesseract.image_to_boxes(gray)
#os.remove(filename)
#print "Text found:\n",text

#print "Data:\n",array
# show the output images

#for entry in data:
    #print "Entry: ", entry
    #cv2.rectangle(gray, (100, 100), (1000, 1000), (0,0,0), 20)
#cv2.namedWindow('Input', cv2.WINDOW_NORMAL)
#cv2.imshow("Input", gray)
cv2.imwrite('Boxes.png',gray)
cv2.imwrite('Estimation.png',newPage)

Text:  6%
ABSA

Transaksie Geskiedenis (2017-12-05 20:40:22)

 

HERBST PROF BM & MEV HMM
15 GROSSKOPFSTRAAT
PAROW

7500

ABSA
3092054062
HUISVERBAND

 

 

 

 

 

 

 

Huidige balans -R 1,190.80
Beskikbare saldo soos op R 279,305.62
Overrekende tjeks R 0.00

Staat vir Periode 2017-11-28 - 2017-12-05

Datum Transaksie Beskrywing

Saldo oorgebring

2017-11-30 DIREKTE KREDIET

2017-12-01 RENTE KAPITALISEE

2017—12-01 DIENSFOOI

2017-12-01 OORDR DT INTERNET

Saldo oorgedra

2017-12-05 08:40:22 bladsy 1 van 1

Bedrag

1,494.29
-11.91
—68.40

-1,ooo.00

Balans

-1 ,604.78
-110.49
-122.40
-190.80

-1 ,190.80

-1,190.80
6   2.53333333333   (159, 458)
%   3.6   (254, 454)
A   2.93333333333   (157, 573)
B   2.83333333333   (250, 571)
S   2.96666666667   (334, 575)
A   2.9   (413, 573)
T   1.1   (124, 629)
r   0.833333333333   (148, 625)
a   0.866666666667   (170, 627)
n   0.8   (197, 625)
s   0.833333333333   (224, 626)
a   0.833333333333   (249, 626)
k   1.1   (276, 630)
s   0.833333333333 

True

In [8]:
##pytesseract.image_to_string??